In [2]:
import  pandas as pd
import recordlinkage
import re
import matplotlib.pyplot as plt

nomba_gtb_fsp = pd.read_csv('Nomba_GTB_BILLS - fsp_16_mar_2024_31_mar_2024.csv', index_col='id_fsp')
nomba_gtb_backend = pd.read_csv('Nomba_GTB_BILLS - Backend_1_Mar_2024_31_2024.csv', index_col='id_backend')

#nomba_gtb_fsp

###Preprocessing 

    1. Ensuring col names in the data sources are different. ( id, transaction date, amount, extracted description) 
    2. Ensuring transaction date is in the same format(no time included).
    3. Extracting description number from both backend and frontend.
    4. Amount in the same format (2d).

In [3]:
nomba_gtb_backend['amount_backend'] = nomba_gtb_backend['amount_backend'].astype(float).map('{:.2f}'.format)
nomba_gtb_fsp['amount_fsp'] = nomba_gtb_fsp['amount_fsp'].astype(float).map('{:.2f}'.format)

nomba_gtb_backend['description_number'] = nomba_gtb_backend['description'].str.split('/').str[0]
#nomba_gtb_backend

In [5]:
def extract_fsp_description_number(text):
    match = re.search(r'TP-COSMIC\s+(\d+)', text)
    return match.group(1) if match else None


nomba_gtb_fsp['description_number_fsp'] = nomba_gtb_fsp['description'].apply(extract_fsp_description_number)
#nomba_gtb_fsp

In [6]:
# Filter for 'bills_payments'
nomba_gtb_fsp_bills_payments = nomba_gtb_fsp[nomba_gtb_fsp['reporting_tag'] == 'bill_payments']

# Filter for 'fees_and_commissions'
nomba_gtb_fsp_fees_and_commissions = nomba_gtb_fsp[nomba_gtb_fsp['reporting_tag'] == 'fees_and_commissions']
#nomba_gtb_fsp

Creating indexer object. Either blocking or full index.
Individual indexer object for comparing fsp to backend and another for backend to fsp

In [7]:
# FSP TO BACKEND
indexer = recordlinkage.Index()
#indexer.block(left_on='transaction_date_fsp',right_on='transaction_date_backend')
#indexer.full()
indexer.block(left_on='description_number_fsp',right_on="description_number")
# BACKEND TO FSP
indexer_backend = recordlinkage.Index()
indexer_backend.block(left_on='transaction_date_backend',right_on='transaction_date_fsp')



<Index>

Blocking on transaction date means only transactions made on the same date are compared.

Incase a transaction date was erroneously recorded on either data source, a false positive / negative is possible

In [8]:
# CANDIDATES FOR FSP TO BACKEND
candidates_combined_fsp = indexer.index(nomba_gtb_fsp, nomba_gtb_backend)
candidates_bills = indexer.index(nomba_gtb_fsp_bills_payments,nomba_gtb_backend)

# CANDIDATES FOR BACKEND TO FSP 
candidates_backend = indexer_backend.index(nomba_gtb_backend, nomba_gtb_fsp)
len(candidates_backend)


# CANDIDATES FOR BACKEND TO FSP bills
candidates_backend_bills = indexer_backend.index(nomba_gtb_backend, nomba_gtb_fsp_bills_payments)
len(candidates_backend_bills)

# CANDIDATES FOR BACKEND TO FSP FEES
candidates_backend_fees = indexer_backend.index(nomba_gtb_backend, nomba_gtb_fsp_fees_and_commissions)
len(candidates_backend_fees)


# full index produces 11m+ candidates

1938315

compare options: exact, string, date , add.

Set threshold and method.

### Comparison methods
    1. Jaro-winkler
    2. Levenshtein
    3. Cosine Similarity
    4. Damerau-Levenshtein
    5. Hamming distance
    6. Soundex - not relevant here
    
Ref : https://www.getcensus.com/blog/implementing-entity-resolution-with-python-record-linkage

In [9]:
# initialize Compare classes
compare_backend = recordlinkage.Compare()
compare_backend_bills = recordlinkage.Compare()
#compare_backend_fees = recordlinkage.Compare()

compare_backend.string("transaction_date_backend","transaction_date_fsp" ,label = 'transaction_date')
compare_backend.exact("amount_backend","amount_fsp",  label = 'amount')

compare_backend_bills.string("transaction_date_backend","transaction_date_fsp" ,label = 'transaction_date')
compare_backend_bills.exact("amount_backend","amount_fsp", label = 'amount')

#compare_backend_fees.string("transaction_date_fsp","transaction_date_backend" ,label = 'transaction_date')
#compare_backend_fees.string("amount_fsp","amount_backend", label = 'amount')

features_backend = compare_backend.compute(candidates_backend,nomba_gtb_backend,nomba_gtb_fsp)


# backend fsp bills
features_backend_bills = compare_backend_bills.compute(candidates_backend_bills,nomba_gtb_backend,nomba_gtb_fsp_bills_payments)

# backend fsp fees
#features_backend_fees = compare_backend_fees.compute(candidates_backend_fees,nomba_gtb_fsp_fees_and_commissions,nomba_gtb_backend)


In [10]:
features_backend['sum']= features_backend['transaction_date'] + features_backend['amount']
features_backend_bills['sum']= features_backend_bills['transaction_date'] + features_backend_bills['amount']
#features_backend_fees['sum']= features_backend_fees['transaction_date'] + features_backend_fees['amount']


In [11]:
#features_backend_fees

In [12]:
features_backend_bills

transaction_date  amount  sum
id_backend id_fsp                                  
684754662  493532329               1.0       0  1.0
           493532155               1.0       0  1.0
           493532407               1.0       0  1.0
           493532377               1.0       0  1.0
           493531853               1.0       0  1.0
...                                ...     ...  ...
684762638  493528081               1.0       0  1.0
           493528051               1.0       0  1.0
           493528427               1.0       0  1.0
           493528251               1.0       0  1.0
           493528175               1.0       0  1.0

[1936834 rows x 3 columns]

In [13]:
features_backend

transaction_date  amount  sum
id_backend id_fsp                                  
684754662  493532329               1.0       0  1.0
           493532155               1.0       0  1.0
           493532407               1.0       0  1.0
           493532377               1.0       0  1.0
           493531853               1.0       0  1.0
...                                ...     ...  ...
684762638  493528476               1.0       0  1.0
           493528478               1.0       0  1.0
           493528480               1.0       0  1.0
           493528482               1.0       0  1.0
           493528484               1.0       0  1.0

[3875988 rows x 3 columns]

In [14]:
best_match_backend = features_backend.loc[features_backend.groupby('id_backend')['sum'].idxmax()]
best_match_backend

,,transaction_date,amount,sum
id_backend,id_fsp,,,
684754594,493531435,1.0,1,2.0
684754595,493531043,1.0,1,2.0
684754596,493531371,1.0,1,2.0
684754597,493530791,1.0,1,2.0
684754598,493531319,1.0,1,2.0
...,...,...,...,...
684765329,493524637,1.0,0,1.0
684765341,493523671,1.0,1,2.0
684765342,493523671,1.0,1,2.0
